While I'm waiting for a full parmaeter cube to be generated, I'm testing the emulation framework I've made so far. /

In [14]:
import numpy as np
from src.emulator import build_emulator, emulate, emulate_wrt_r
from src.allCorrFunc import RBINS
from src.doBatchCalls import BOUNDS, N_PER_DIM

In [15]:
for key, val in BOUNDS.iteritems():
    print key
    print np.linspace(val[0], val[1], num=N_PER_DIM)
    print '*'*30

logMmin
[ 11.7         11.96666667  12.23333333  12.5       ]
******************************
f_c
[ 0.1         0.23333333  0.36666667  0.5       ]
******************************
logM0
[ 10.  11.  12.  13.]
******************************
sigma_logM
[ 0.2         0.36666667  0.53333333  0.7       ]
******************************
logM1
[ 13.1  13.5  13.9  14.3]
******************************
alpha
[ 0.75        0.91666667  1.08333333  1.25      ]
******************************


In [16]:
y_param = 'logMmin'
#y_param = 'f_c'
#y_param = 'logM0'
#y_param = 'logM1'
#y_param = 'sigma_logM'
#y_param = 'alpha'

#param_2 = 'logMmin'
#param_2 = 'sigma_logM'
param_2 = 'f_c'

param_3 = 'sigma_logM'

#param_4 = 'alpha'

#param_5= 'logM1'

emulation_point = [('f_c', 0.233), 
                   ('logM0', 12.0), ('sigma_logM', 0.533), ('alpha', 1.083),
                   ('logM1', 13.5), ('logMmin', 12.233)]

fiducial_point = {'logM0': 12.20, 'logM1': 13.7, 'alpha': 1.02, 
                  'logMmin': 12.1, 'f_c': 0.19, 'sigma_logM': 0.46}
fixed_params = {key:val for key, val in emulation_point}
#em_params = {}
em_params = {param_2:fixed_params[param_2], param_3:fixed_params[param_3],
            param_4:fixed_params[param_4]}#, param_5:fixed_params[param_5]} #varying y_param, holding this fixed in the emulator now. 
del fixed_params[y_param]
del fixed_params[param_2]
del fixed_params[param_3]
#del fixed_params[param_4]
#del fixed_params[param_5]

In [17]:
#%%timeit
from time import time
t0 = time()
gp, log_xi, log_xi_cov = build_emulator(fixed_params)
print '%.3f s'%(time()-t0)

64 Points used for training.
GP Params:  [ 0.30783176  0.08759131  0.03091122  0.04489427  0.03159018]
59.755 s


In [18]:
from scipy.linalg import eigvals, det
print log_xi_cov[0,0]
print det(log_xi_cov/log_xi_cov[0,0])

2.21719516101e-05
0.0


In [19]:
c = log_xi_cov[:18,:][:,:18]
for e in eigvals(c):
    print e, e > 0

(0.00247670680274+0j) True
(3.10770144589e-05+0j) True
(2.32709290819e-05+0j) True
(1.91845702425e-05+0j) True
(1.08450867046e-05+0j) True
(6.56558358882e-06+0j) True
(3.92984430191e-06+0j) True
(1.91468733092e-06+0j) True
(1.46637760929e-06+0j) True
(1.05184225457e-06+0j) True
(8.76785394195e-07+0j) True
(7.07172376322e-07+0j) True
(6.19854482869e-07+0j) True
(3.54710819218e-07+0j) True
(1.57281943766e-07+0j) True
(2.86771169933e-07+0j) True
(2.22903752243e-07+0j) True
(2.53549570389e-07+0j) True


In [20]:
log_xi_mean = log_xi.mean()
log_xi-=log_xi_mean

In [21]:
#gp.kernel[:] = [0.175, -0.32, -3.4]

In [22]:
yp = np.linspace(BOUNDS[y_param][0],BOUNDS[y_param][1], num=N_PER_DIM)
N = 200
rpoints = np.linspace(-1, 1.5, N)
RBIN_CENTERS = ((RBINS[:-1]+RBINS[1:])/2)
outputs = emulate_wrt_r(gp, log_xi, em_params, rpoints,
                                    y_param=y_param,y_points=yp)

AssertionError: 

In [ ]:
from matplotlib import pyplot as plt
from itertools import cycle
%matplotlib inline
#plt.style.use('fivethirtyeight')
import seaborn as sns
sns.set()
colors = cycle(sns.color_palette())

In [ ]:
print log_xi.shape

In [ ]:
#note should write a function that gives plot_xi given em_params
plot_xi = 10**(log_xi+log_xi_mean)
plot_xi = plot_xi.reshape((N_PER_DIM**(1+len(em_params)),len(RBINS)-1))

plot_xi_err = np.sqrt(np.diag(log_xi_cov))
plot_xi_err = plot_xi_err.reshape((N_PER_DIM**(1+len(em_params)),len(RBINS)-1) )

plot_xi_err*=plot_xi*np.log(10)

if len(em_params) > 0:
    #plot_xi = plot_xi[2*N_PER_DIM:3*N_PER_DIM, :]
    plot_xi = plot_xi[2:-1:N_PER_DIM, :]
    
    #plot_xi_err = plot_xi_err[2*N_PER_DIM:3*N_PER_DIM, :]
    plot_xi_err = plot_xi_err[2:-1:N_PER_DIM, :]

In [ ]:
fig = plt.figure(figsize = (10, 10))
for label, (log_xi_pred, log_xi_err),px, err,c in zip(yp, outputs, plot_xi,plot_xi_err,colors):

    plt.plot(10**rpoints, 10**(log_xi_pred+log_xi_mean),label='%.3f'%label,color = c)
    #plt.fill_between(10**rpoints,10**(log_xi_pred+log_xi_mean+log_xi_err),
    #                 10**(log_xi_pred+log_xi_mean-log_xi_err), color = c, alpha = 0.3)
    
    plt.fill_between(10**rpoints,10**(log_xi_pred+log_xi_mean)*(1+np.log(10)*log_xi_err),
                     10**(log_xi_pred+log_xi_mean)*(1-np.log(10)*log_xi_err), color = c, alpha = 0.3)

    #plt.errorbar(RBIN_CENTERS, px,err, color =c,fmt = 'o')
    
plt.xlim(xmin = 0.1, xmax = 30)
plt.ylim(ymin = 1e-2, ymax = 1e4)
plt.title("Emulator w.r.t. %s"%y_param, fontsize =20)
plt.loglog()
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel(r'$r $  $\rm{[Mpc]}$', fontsize=25)
plt.ylabel(r'$\xi_{\rm gg}(r)$', fontsize=25)
plt.legend(loc='best', fontsize=20)  

In [ ]:
N = 200
plot_xi = plot_xi.T #switch rows and cols
plot_xi_err = plot_xi_err.T
yp = np.linspace(BOUNDS[y_param][0],BOUNDS[y_param][1], num=N)
plot_yp = np.linspace(BOUNDS[y_param][0],BOUNDS[y_param][1], num=N_PER_DIM)
rpoints = np.log10(RBIN_CENTERS)
#print rpoints
outputs = emulate(gp, log_xi, em_params,x_param=y_param, x_points=yp,y_param='r',y_points=rpoints)

In [ ]:
i = 4
#remove some points
rpoints=rpoints[::i]
outputs=outputs[::i]
plot_xi=plot_xi[::i]
plot_xi_err = plot_xi_err[::i]

In [ ]:
fig = plt.figure(figsize = (10, 10))

for label, (log_xi_pred, log_xi_err),px,err,c in zip(rpoints, outputs, plot_xi,plot_xi_err,colors):

    plt.plot(yp, 10**(log_xi_pred+log_xi_mean),label='%.3f Mpc'%(10**label),color = c)
    
    plt.fill_between(yp,10**(log_xi_pred+log_xi_mean)*(1+np.log(10)*log_xi_err),
                     10**(log_xi_pred+log_xi_mean)*(1-np.log(10)*log_xi_err), color = c, alpha = 0.3)
    #plt.errorbar(plot_yp, px,err, color =c,fmt = 'o')
    
plt.xlim(xmin = 0.97*BOUNDS[y_param][0], xmax = BOUNDS[y_param][1]*1.03)
plt.ylim(ymin = 10**(-2.5), ymax = 10**(3.5))
plt.title("Emulator w.r.t. %s"%y_param, fontsize =20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.yscale('log')
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel(r'$\sigma_{log(M)}$  $\rm{[Log_{10} M_{\odot}]}$', fontsize=25)
#plt.xlabel(r'$\alpha$ ', fontsize=25)
plt.ylabel(r'$\xi_{\rm gg}(r)$', fontsize=25)
plt.legend(loc='best', fontsize=20)  